# 获取12306站点数据（Python爬取）
    # 思路分析：
        # 1. 爬取出发地、目的地 - Code映射（该部分数据可以持久化，因为映射关系基本不会变动）
        # 2. 根据车次，获取到 train_no。例如，获取到 G323 的train_no为 240000G3230A
        # 3. 根据train_no、出发地code、目的地code、日期获取经过站点信息
        # 4. 弊端：
            #    （1）爬取数据源来自12306，只能获取到有效日期的车次信息（即，历史车次信息查询不到）
            #    （2）脚本还不够智能，例如，执行速度、批量爬取等

In [63]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json

## 1. 爬取“站点-code”映射关系数据（json）

In [64]:
# 根据查看源码查出来的
url = 'https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9017' 
response = requests.get(url,verify=False)  
# 用正则表达式 来获取车站的拼音和大小写字母的代号信息
stations = re.findall(u'([\u4e00-\u9fa5]+)\|([A-Z]+)',response.text)
# 站点-code 映射关系,打印前 10 条
stations[:10]

C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[('北京北', 'VAP'),
 ('北京东', 'BOP'),
 ('北京', 'BJP'),
 ('北京南', 'VNP'),
 ('北京西', 'BXP'),
 ('广州南', 'IZQ'),
 ('重庆北', 'CUW'),
 ('重庆', 'CQW'),
 ('重庆南', 'CRW'),
 ('广州东', 'GGQ')]

## 2. 站点-code 映射关系转化，并打印前 10 条

In [66]:
df = pd.DataFrame(list(stations), columns=['station_key', 'station_value'])
df.head(10)

,station_key,station_value
0,北京北,VAP
1,北京东,BOP
2,北京,BJP
3,北京南,VNP
4,北京西,BXP
5,广州南,IZQ
6,重庆北,CUW
7,重庆,CQW
8,重庆南,CRW
9,广州东,GGQ


In [67]:
df.to_csv("C:\\Users\\syndt\\Desktop\\stations_mapping.csv", index=False)

## [注] 订票查询初始信息--（日期、起始地、目的地）、车次（可选）

In [68]:
# 查询 日期（给出）
#================================================================
depart_date = '2017-07-06'
# 出发地
from_station_name = '北京'
# 目的地
to_station_name = '厦门'
#================================================================

# 1. 数据转化为字典，从而获取 to/from_station_telecode
dct = dict(stations)
from_station_telecode = dct[from_station_name]
to_station_telecode = dct[to_station_name]
url = 'https://kyfw.12306.cn/otn/leftTicket/query?leftTicketDTO.train_date=' + depart_date + '&leftTicketDTO.from_station=' + from_station_telecode + '&leftTicketDTO.to_station=' + to_station_telecode + '&purpose_codes=ADULT'
respt = requests.get(url,verify=False)  

# 2. 数据转化为字典，从而获取 train_no
stat = {i[1] : i[0] for i in re.findall(u'预定|([A-Z0-9]{12})\|([A-Z]{1}[0-9]{1,6})',respt.text)}

# 指定列车号（可以查询单个、数组循环多个）
#================================================================
train_num = 'G323'
#================================================================
# 得到 对应 train_no
train_no = stat[train_num]

C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## 3. 指定车次、时间，获取数据源，的所经站点信息

In [130]:
# 获取源数据（12306）
url = 'https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no=' + train_no + '&from_station_telecode=' + from_station_telecode + '&to_station_telecode=' + to_station_telecode + '&depart_date=' + depart_date 
resp = requests.get(url,verify=False)  

# 数据加工、处理
station_no = pd.DataFrame(re.findall(u'"station_no":"([0-9]{1,2})"',resp.text), columns=['站序'])
station_name = pd.DataFrame(re.findall(u'"station_name":"([\u4e00-\u9fa5]+)"',resp.text), columns=['站名'])
arrive_time = pd.DataFrame(re.findall(u'"arrive_time":"(-{0,4}[0-9]{0,2}:{0,1}[0-9]{0,2})"',resp.text), columns=['到站时间'])
start_time = pd.DataFrame(re.findall(u'"start_time":"(-{0,4}[0-9]{0,2}:{0,1}[0-9]{0,2})"',resp.text), columns=['出发时间'])
stopover_time = pd.DataFrame(re.findall(u'"stopover_time":"(-{0,4}[0-9]{0,5}[\u4e00-\u9fa5]{0,10})"',resp.text), columns=['停留时间'])

C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## 4. 连接数据并输出

In [132]:
final_df.to_csv("C:\\Users\\syndt\\Desktop\\final_df.csv", index=False)

In [131]:
final_df = station_no.join(station_name).join(arrive_time).join(start_time).join(stopover_time)
final_df

,站序,站名,到站时间,出发时间,停留时间
0,01,北京南,----,10:10,----
1,02,廊坊,10:31,10:33,2分钟
2,03,沧州西,11:08,11:10,2分钟
3,04,济南西,11:56,11:58,2分钟
4,05,泰安,12:15,12:17,2分钟
5,06,滕州东,12:48,12:50,2分钟
6,07,徐州东,13:15,13:17,2分钟
7,08,蚌埠南,13:54,13:56,2分钟
8,09,水家湖,14:16,14:18,2分钟
9,10,合肥南,14:52,14:59,7分钟


In [134]:
final_df.describe()

,站序,站名,到站时间,出发时间,停留时间
count,30,30,30,30,30
unique,30,30,30,30,6
top,13,沧州西,21:12,17:48,2分钟
freq,1,1,1,1,21


In [136]:
## 知识点总结：
    # 1. 正则表达式
    #     例如，某一列的数据类型为如下格式：
    #         s = '你好----'
    #         s1 = '你好1234'
    #     即，过滤出“你好”后面的字符。匹配规则：re.findall(u'你好(-{0,4}[0-9]{0,4})', s)
    # 2. 参考文章：
    #     http://blog.csdn.net/claram/article/details/54633113